### gseapy analysis of Velos_AW_20170831
gsea algorithm: http://software.broadinstitute.org/gsea/doc/GSEAUserGuideFrame.html  
gseapy: https://media.readthedocs.org/pdf/gseapy/latest/gseapy.pdf  


In [3]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
pd.set_option('display.max_colwidth', -1)
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2018-01-04) 49,283 GO Terms


In [6]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = 'out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_annot.pkl.gz"))

In [10]:
def make_go2Gene_map(grouped_loci, ontology='MF'):    

    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return dict(out)

def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    to_keep = set()
    for brother in go_locus.keys():
        to_keep.add(brother) # make sure filtered out siblings don't filter out ones we want to keep
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            if brother in siblings:
                siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling] and not sibling in to_keep:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [11]:
def run_go_gsea(rank_df, g2g_map, seed, outdir='tmp'):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir=outdir, min_size = 5, max_size=500, 
                     permutation_n = 10000, graph_num = 20, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

def plot_gsea_result(row, rank):
    return gp.plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [12]:
mf_map = make_go2Gene_map(grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

bp_map = make_go2Gene_map(grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

cc_map = make_go2Gene_map(grouped_loci, 'CC')
cc_map_f = filter_go2gene_map(cc_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(cc_map), len(cc_map_f)))

Unfiltered: 479	Filtered: 245
Unfiltered: 551	Filtered: 240
Unfiltered: 78	Filtered: 43


### probe_unenriched_gsea

In [20]:
out_dir = "probe_unenriched_gsea"
df = pd.read_csv(os.path.join(BASE,"probe_unenriched_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [21]:
rank_df.head()

,gene_name,rank
0,61499602,-14.121
1,61484779,-12.873
2,22471287,-10.356
3,17773601,-9.807
4,13554466,-9.759


In [22]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2018-01-05 16:15:41,268 Parsing data files for GSEA.............................
/home/gstupp/.local/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2018-01-05 16:15:42,393 0089 gene_sets used for further statistical testing.....
2018-01-05 16:15:42,394 Start to run GSEA...Might take a while..................
2018-01-05 16:26:58,173 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2018-01-05 16:27:05,570 Congratulations...GSEAPY run successfully...............
2018-01-05 16:27:05,593 Parsing data files for GSEA.............................
2018-01-05 16:27:06,575 0101 gene_sets used for further 

In [ ]:
# rt is increased in VK samples == positive FC

In [23]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016781,0.379,4.344,0.000e+00,0.000e+00,333,165,"[38559827, 63600974, 51283234, 58163687, 66542948, 46823448, 44507319, 11783818, 46765848, 30295347, 38724617, 61697822, 37193889, 43255017, 61857832, 11783819, 45352916, 59159610, 46261392, 62544990, 63360902, 38639771, 36779731, 59590660, 61682022, 62903914, 45365445, 56043654, 65509964, 46564018, 51881412, 63168915, 46980869, 39756635, 40968297, 62850845, 43269453, 7261272, 62139199, 66064320, 61717374, 6107407, 12243916, 4854118, 62872850, 61719792, 67203088, 51077372, 5053713, 11790449, 7554865, 18134283, 61704804, 67587599, 39053068, 61686338, 61734588, 167971656, 1963894, 55837820, 78381698, 166397107, 10630096, 18677776, 66752524, 20651923, 20356522, 18672118, 18127587, 11005782, 12680181, 20874127, 15339324, 7967270, 15038592, 7524160, 17395074, 8042238, 3902762, 7200027, 2105368, 2749207, 3710169, 8955380, 4246480, 168149697, 4841345, 166283021, 62771938, 21265599, 61731176, 61723254, 61733588, 10296556, 54203018, 33209292, 6101228, 32633754, 32548173, 30581852, ...]","phosphotransferase activity, paired acceptors"
GO:0046983,0.713,4.132,0.000e+00,0.000e+00,74,23,"[45545712, 49537560, 48905914, 42278268, 63693264, 65206255, 27851587, 20753809, 63996097, 15748223, 63603912, 2057470, 68051075, 209005, 31254632, 11559291, 63132765, 19462566, 31060890, 167668878, 64735529, 53521585, 27658376]",protein dimerization activity
GO:0009055,0.622,3.714,0.000e+00,0.000e+00,64,25,"[11513369, 68130519, 68015005, 63911014, 37627930, 63594929, 66133864, 64317423, 64498103, 20897862, 7171877, 64094124, 63922980, 10796453, 53825558, 64503571, 64001832, 68568418, 7289884, 57375765, 64320533, 61957888, 62726174, 167858203, 63535674]",electron transfer activity
GO:0016301,0.299,3.325,0.000e+00,0.000e+00,474,202,"[14006785, 30875488, 17585926, 38559827, 63600974, 8107297, 45207415, 4543087, 51283234, 52607938, 67028544, 33420375, 67170377, 57780184, 38809343, 44106061, 44507319, 46765848, 30295347, 65807944, 53739636, 52199423, 60544652, 38724617, 61697822, 45694911, 45109698, 36464393, 41728649, 43255017, 61857832, 11783819, 14714199, 40688290, 40877584, 46812465, 45352916, 68897404, 46261392, 62407115, 41095919, 62381609, 62544990, 55043981, 59177947, 63360902, 38639771, 61682022, 62903914, 45365445, 65509964, 46564018, 59658213, 7261272, 62139199, 59643079, 61717374, 62032555, 6107407, 64396107, 58799782, 12243916, 4854118, 62670158, 61719792, 5053713, 7554865, 18134283, 61704804, 67587599, 64512511, 39053068, 61686338, 61734588, 167971656, 51320879, 166287375, 59640160, 61355398, 51389790, 1963894, 78381698, 10630096, 18677776, 66752524, 20651923, 20356522, 18672118, 18127587, 11005782, 12680181, 20874127, 15339324, 7967270, 15038592, 7524160, 17395074, 8042238, 3902762, 7200027, ...]",kinase activity
GO:0016774,0.417,3.301,0.000e+00,0.000e+00,147,47,"[14006785, 30875488, 17585926, 8107297, 45207415, 4543087, 67028544, 33420375, 67170377, 57780184, 38809343, 44106061, 65807944, 53739636, 52199423, 45694911, 45109698, 36464393, 41728649, 40877584, 46812465, 68897404, 62407115, 41095919, 59177947, 59658213, 62032555, 64396107, 58799782, 62670158, 66422871, 36790151, 60180834, 167977924, 62931402, 15749576, 38944579, 13193055, 52066393, 43225398, 36495567, 63163362, 5201609, 53974684, 28225703, 165910996, 47719952]","phosphotransferase activity, carboxyl group as acceptor"
GO:0004807,0.635,3.274,0.000e+00,0.000e+00,47,18,"[70998800, 45872548, 41483770, 20059228, 42577952, 29514768, 10323867, 42871001, 59429091, 56895760, 62032555, 60154270, 65210690, 37609477, 38730482, 18678412, 30856007, 21331797]",triose-phosphate isomerase activity
GO:0016866,0.485,3.183,0.000e+00,0.000e+00,72,29,"[47525739, 60946261, 56498826, 55558209, 47355983, 52806950, 39584199, 48680784, 62031992, 44538085, 62215888, 62101856, 62236100, 15750195, 56897671, 39173991, 33141595, 45539402, 48009305, 62686112, 50672349, 37626150, 42379485, 8454985,

In [24]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0060089,-0.528,-2.903,0.000e+00,0.000e+00,86,76,"[49662133, 13560030, 37017841, 37761511, 58552473, 48256877, 55935267, 41809956, 39468098, 53918604, 42005004, 56581658, 61686560, 36220708, 55929673, 45394073, 47572664, 61660972, 53056830, 47506344, 41788061, 38737848, 50420785, 57861457, 63778490, 14080080, 37455868, 60539194, 38135581, 59048929, 43782161, 59355757, 45663281, 60812906, 55654955, 61625425, 59653821, 61633377, 49788684, 55924498, 61641119, 63844951, 41281246, 61586644, 39070204, 50542139, 52806222, 46890852, 44791733, 42391645, 61638866, 44928939, 167650280, 43772930, 49582748, 56469614, 47140453, 45646458, 44923942, 33139570, 59052976, 61260585, 61638521, 42336448, 61670085, 61669286, 36279988, 61618580, 61630187, 61644628, 52808333, 61631911, 21416701, 61687390, 61647961, 61656451]",molecular transducer activity
GO:0004872,-0.528,-2.901,0.000e+00,0.000e+00,85,76,"[49662133, 13560030, 37017841, 37761511, 58552473, 48256877, 55935267, 41809956, 39468098, 53918604, 42005004, 56581658, 61686560, 36220708, 55929673, 45394073, 47572664, 61660972, 53056830, 47506344, 41788061, 38737848, 50420785, 57861457, 63778490, 14080080, 37455868, 60539194, 38135581, 59048929, 43782161, 59355757, 45663281, 60812906, 55654955, 61625425, 59653821, 61633377, 49788684, 55924498, 61641119, 63844951, 41281246, 61586644, 39070204, 50542139, 52806222, 46890852, 44791733, 42391645, 61638866, 44928939, 167650280, 43772930, 49582748, 56469614, 47140453, 45646458, 44923942, 33139570, 59052976, 61260585, 61638521, 42336448, 61670085, 61669286, 36279988, 61618580, 61630187, 61644628, 52808333, 61631911, 21416701, 61687390, 61647961, 61656451]",receptor activity
GO:0005215,-0.392,-2.516,0.000e+00,0.000e+00,408,251,"[22519274, 40719712, 66122622, 62209139, 38057567, 61978811, 55065338, 56276091, 65738732, 41603471, 62257631, 59151898, 47067501, 61937198, 65553828, 50248755, 62284628, 58453185, 37670366, 49630873, 43379581, 56120244, 45151444, 57089264, 65272814, 62545344, 63468226, 38031187, 40686252, 40726507, 39569838, 62360359, 53113770, 62883525, 40659878, 43232184, 63133221, 57110947, 63132923, 28669498, 57638373, 66502215, 63425417, 61017707, 45702310, 64694136, 9505311, 66703872, 28273397, 61670344, 43103519, 57584638, 56215621, 57118414, 47262337, 37969849, 66402745, 61865500, 29371220, 63078357, 38018722, 49662133, 65627086, 51679236, 13560030, 62354250, 53957349, 62415539, 37017841, 66602623, 40085118, 37761511, 58552473, 36311677, 58930696, 53065729, 62603736, 59408423, 52294946, 48256877, 62187193, 63423769, 74407057, 10791111, 62687900, 62121153, 55935267, 60506620, 47816374, 46089913, 41809956, 62665934, 62521088, 37270167, 62659413, 40273714, 39468098, 53918604, 42005004, 41466580, ...]",transporter activity
GO:0030246,-0.538,-2.293,1.130e-04,2.326e-04,41,26,"[46103326, 49527216, 21250111, 51873873, 63509442, 63549178, 54520227, 38215699, 50452418, 42544980, 66779826, 45897671, 44719990, 48607373, 61569205, 60193447, 37947140, 46208104, 54169939, 62261302, 61578978, 61691554, 61587239, 61876717, 61565673, 61637435]",carbohydrate binding
GO:0003938,-0.438,-2.279,0.000e+00,2.080e-04,82,59,"[27851198, 49839779, 62162256, 11552150, 6168952, 21398217, 20636024, 18114733, 11074403, 167011134, 13079751, 17595180, 12683022, 17638464, 5750132, 7298315, 5268480, 8163776, 9110582, 9598464, 10448361, 6125354, 12246457, 6102039, 16993373, 4237753, 17198522, 5250429, 18591605, 21628733, 62540505, 28409016, 23047997, 21662701, 21657012, 62408953, 56812422, 6851580, 17401453, 22700445, 5245426, 62365178, 62511612, 65475069, 62525598, 4681425, 14378567, 18588572, 2559454, 2078088, 6105533, 6317552, 18676616, 62285636, 4672299, 30072883, 20852621, 29213638, 11807821]",IMP dehydrogenase activity
GO:0008135,-0.342,-2.051,0.000e+00,2.472e-03,234,131,"[47627272, 10185112, 5730052, 42899619, 64639435, 22603881, 39547316, 44791138, 63944471, 63541038, 64773746, 16613806, 12268875, 2

In [25]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0043648,0.607,3.228,0.000e+00,0.000,48,19,"[22471287, 13684061, 6155443, 25922093, 66992387, 11147909, 18316344, 5269880, 63627128, 31254632, 11559291, 63132765, 31060890, 167668878, 22083024, 46209639, 63613061, 27658376, 62814802]",dicarboxylic acid metabolic process
GO:0044262,0.411,2.556,0.000e+00,0.002,102,27,"[13684061, 6155443, 66992387, 67211017, 11147909, 18316344, 5269880, 68559205, 69590063, 53512330, 46339502, 63627128, 65683090, 41723985, 63393891, 28898822, 3991754, 66652236, 66845691, 48678785, 41891564, 63156937, 36117541, 63342186, 62636502, 62405972, 38175368]",cellular carbohydrate metabolic process
GO:1901361,0.592,2.495,4.869e-04,0.002,44,12,"[61781566, 41733069, 36166078, 65471645, 60329713, 37106339, 36781703, 51595459, 50881124, 60969027, 31255758, 62013137]",organic cyclic compound catabolic process
GO:0019439,0.592,2.477,4.675e-04,0.002,44,12,"[61781566, 41733069, 36166078, 65471645, 60329713, 37106339, 36781703, 51595459, 50881124, 60969027, 31255758, 62013137]",aromatic compound catabolic process
GO:0009132,0.229,2.293,0.000e+00,0.003,234,86,"[14006785, 30875488, 17585926, 8107297, 45207415, 4543087, 67028544, 33420375, 12856286, 29629717, 65807944, 53739636, 52199423, 60544652, 47046945, 45694911, 45109698, 46513244, 36464393, 41728649, 14714199, 40688290, 40877584, 46812465, 68897404, 29514768, 10323867, 62407115, 41095919, 28234013, 55043981, 59177947, 63649356, 59658213, 15986841, 59643079, 62236100, 62032555, 60931305, 57027218, 20139794, 17549174, 38603820, 62670158, 62165325, 42399038, 43189595, 30856007, 51320879, 166287375, 59640160, 61355398, 51389790, 44448128, 57125904, 57468614, 8454985, 13441968, 66422871, 47499867, 36790151, 60180834, 167977924, 20910032, 62382413, 63770542, 19344501, 37284587, 15749576, 64479173, 38944579, 13193055, 36495567, 63163362, 51094178, 5201609, 11641787, 53974684, 65447256, 58995018, 28225703, 165910996, 49334630, 64427981, 13180694, 47719952]",nucleoside diphosphate metabolic process
GO:0072524,0.225,2.210,0.000e+00,0.005,240,88,"[14006785, 30875488, 17585926, 8107297, 45207415, 4543087, 67028544, 33420375, 12856286, 29629717, 65807944, 53739636, 52199423, 60544652, 47046945, 45694911, 45109698, 46513244, 36464393, 41728649, 14714199, 40688290, 40877584, 46812465, 68897404, 29514768, 10323867, 62407115, 41095919, 28234013, 55043981, 59177947, 63649356, 59658213, 15986841, 59643079, 66876017, 62236100, 62032555, 63282343, 60931305, 57027218, 20139794, 17549174, 38603820, 62670158, 62165325, 42399038, 43189595, 30856007, 51320879, 166287375, 59640160, 61355398, 51389790, 44448128, 57125904, 57468614, 8454985, 13441968, 66422871, 47499867, 36790151, 60180834, 167977924, 20910032, 62382413, 63770542, 19344501, 37284587, 15749576, 64479173, 38944579, 13193055, 36495567, 63163362, 51094178, 5201609, 11641787, 53974684, 65447256, 58995018, 28225703, 165910996, 49334630, 64427981, 13180694, 47719952]",pyridine-containing compound metabolic process
GO:0046939,0.229,2.192,0.000e+00,0.005,234,86,"[14006785, 30875488, 17585926, 8107297, 45207415, 4543087, 67028544, 33420375, 12856286, 29629717, 65807944, 53739636, 52199423, 60544652, 47046945, 45694911, 45109698, 46513244, 36464393, 41728649, 14714199, 40688290, 40877584, 46812465, 68897404, 29514768, 10323867, 62407115, 41095919, 28234013, 55043981, 59177947, 63649356, 59658213, 15986841, 59643079, 62236100, 62032555, 60931305, 57027218, 20139794, 17549174, 38603820, 62670158, 62165325, 42399038, 43189595, 30856007, 51320879, 166287375, 59640160, 61355398, 51389790, 44448128, 57125904, 57468614, 8454985, 13441968, 66422871, 47499867, 36790151, 60180834, 167977924, 20910032, 62382413, 63770542, 19344501, 37284587, 15749576, 64479173, 38944579, 13193055, 36495567, 63163362, 51094178, 5201609, 11641787, 53974684, 65447256, 58995018, 28225703, 165910996, 49334630, 64427981, 13180694, 47719952]",nucleotide phosphorylation
GO:0006733,0.225,2.172,0.000e+

In [26]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0006928,-0.641,-4.029,0.000e+00,0.000e+00,192,192,"[67153300, 64393106, 63578755, 52493103, 46374759, 63120114, 44786243, 63423978, 38387187, 43234476, 13586933, 63285308, 63543912, 37969809, 63436090, 48252443, 57751592, 62964712, 62062741, 62981262, 168017786, 62217709, 167570053, 166894504, 45075220, 51386722, 64167744, 51263942, 51837448, 64297496, 69175457, 65866265, 22143824, 62454720, 167732614, 167602424, 168130942, 167560438, 167355590, 167344702, 167246340, 166985137, 167799665, 67218229, 64780867, 63954167, 63277926, 62966887, 62682149, 62619244, 57528177, 49587153, 13589494, 43280069, 13571055, 167601916, 62560831, 62613597, 64917247, 166208736, 61924219, 62002315, 54589912, 62473024, 63356547, 62641817, 62699037, 62769590, 62263580, 63721498, 21809431, 62544664, 62303301, 3775835, 63636819, 55984017, 63842059, 63196643, 62266334, 63373149, 62570953, 62624850, 62530384, 62753172, 62784306, 62880763, 62891528, 62713836, 55916126, 62259838, 62626786, 62451969, 62525835, 62853162, 22310619, 62543065, 62581410, 167949258, 62540834, 20837867, ...]",movement of cell or subcellular component
GO:0040011,-0.641,-4.015,0.000e+00,0.000e+00,192,192,"[67153300, 64393106, 63578755, 52493103, 46374759, 63120114, 44786243, 63423978, 38387187, 43234476, 13586933, 63285308, 63543912, 37969809, 63436090, 48252443, 57751592, 62964712, 62062741, 62981262, 168017786, 62217709, 167570053, 166894504, 45075220, 51386722, 64167744, 51263942, 51837448, 64297496, 69175457, 65866265, 22143824, 62454720, 167732614, 167602424, 168130942, 167560438, 167355590, 167344702, 167246340, 166985137, 167799665, 67218229, 64780867, 63954167, 63277926, 62966887, 62682149, 62619244, 57528177, 49587153, 13589494, 43280069, 13571055, 167601916, 62560831, 62613597, 64917247, 166208736, 61924219, 62002315, 54589912, 62473024, 63356547, 62641817, 62699037, 62769590, 62263580, 63721498, 21809431, 62544664, 62303301, 3775835, 63636819, 55984017, 63842059, 63196643, 62266334, 63373149, 62570953, 62624850, 62530384, 62753172, 62784306, 62880763, 62891528, 62713836, 55916126, 62259838, 62626786, 62451969, 62525835, 62853162, 22310619, 62543065, 62581410, 167949258, 62540834, 20837867, ...]",locomotion
GO:0042026,-0.533,-2.726,0.000e+00,0.000e+00,79,53,"[36306027, 48102552, 21210212, 8541817, 64990955, 14914677, 36473885, 63403497, 40486721, 28274786, 50281676, 53554021, 5710574, 64285414, 28150928, 63523057, 62297581, 64993744, 4693399, 66285314, 5495963, 3713116, 65149315, 62320869, 16738939, 7298179, 4205147, 29187158, 42403062, 7439957, 12255964, 4663007, 166815704, 64901603, 4096188, 7694238, 50356580, 20870932, 66734155, 38842066, 165824520, 62282085, 15046618, 58886412, 65231447, 15010122, 40203993, 44414346, 60403481, 65051813, 50135812, 65789189, 37978224]",protein refolding
GO:0051179,-0.380,-2.467,0.000e+00,1.570e-05,477,282,"[22519274, 40719712, 66122622, 62209139, 38057567, 61978811, 55065338, 56276091, 65738732, 41603471, 62257631, 59151898, 47067501, 61937198, 65553828, 50248755, 62284628, 58453185, 37670366, 49630873, 62643753, 43379581, 56120244, 45151444, 57089264, 65272814, 62545344, 63468226, 38031187, 40686252, 40726507, 39569838, 62360359, 53113770, 62883525, 64163466, 40659878, 43232184, 63133221, 57110947, 63132923, 28669498, 57638373, 61607570, 66502215, 63425417, 61017707, 45702310, 55975632, 64694136, 38182866, 9505311, 66703872, 46574529, 28273397, 61670344, 43103519, 57584638, 56215621, 57118414, 47262337, 37969849, 66402745, 61865500, 29371220, 63078357, 38018722, 49662133, 11403880, 55289846, 41547405, 65627086, 62649376, 54721029, 51679236, 13560030, 62354250, 53957349, 62415539, 37017841, 66602623, 38164354, 40085118, 37761511, 58552473, 36311677, 58930696, 53065729, 62603736, 59408423, 52294946, 48256877, 62187193, 63423769, 74407057, 10791111, 62687900, 27529220, 62121153, 55935267, ...]",localization
GO:0051234,-0.379,-2.466,0.000e+00,1.256e-05,476,281,"[22519274

In [27]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [28]:
cc_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044463,-0.678,-4.207,0.000e+00,0.000e+00,167,167,"[64393106, 63578755, 52493103, 46374759, 63120114, 13586933, 63285308, 37969809, 63436090, 62964712, 62062741, 62981262, 168017786, 62217709, 167570053, 166894504, 51386722, 51837448, 64297496, 65866265, 22143824, 62454720, 167732614, 167602424, 168130942, 167560438, 167355590, 167344702, 167246340, 166985137, 167799665, 67218229, 64780867, 63954167, 63277926, 62966887, 62682149, 62619244, 57528177, 49587153, 13589494, 43280069, 13571055, 167601916, 62560831, 62613597, 64917247, 166208736, 61924219, 62002315, 54589912, 62473024, 63356547, 62641817, 62699037, 62769590, 62263580, 63721498, 21809431, 62544664, 62303301, 3775835, 63636819, 55984017, 63196643, 62266334, 63373149, 62570953, 62624850, 62530384, 62753172, 62784306, 62880763, 62891528, 62713836, 55916126, 62259838, 62626786, 62451969, 62525835, 62853162, 22310619, 62543065, 62581410, 167949258, 62540834, 20837867, 62408493, 65774791, 62412749, 62566843, 62881914, 63245372, 62097905, 63669394, 62803002, 62582614, 62680657, 62719918, 37195121, ...]",cell projection part
GO:0042995,-0.641,-4.027,0.000e+00,0.000e+00,195,192,"[67153300, 64393106, 63578755, 52493103, 46374759, 63120114, 44786243, 63423978, 38387187, 43234476, 13586933, 63285308, 63543912, 37969809, 63436090, 48252443, 57751592, 62964712, 62062741, 62981262, 168017786, 62217709, 167570053, 166894504, 45075220, 51386722, 64167744, 51263942, 51837448, 64297496, 69175457, 65866265, 22143824, 62454720, 167732614, 167602424, 168130942, 167560438, 167355590, 167344702, 167246340, 166985137, 167799665, 67218229, 64780867, 63954167, 63277926, 62966887, 62682149, 62619244, 57528177, 49587153, 13589494, 43280069, 13571055, 167601916, 62560831, 62613597, 64917247, 166208736, 61924219, 62002315, 54589912, 62473024, 63356547, 62641817, 62699037, 62769590, 62263580, 63721498, 21809431, 62544664, 62303301, 3775835, 63636819, 55984017, 63842059, 63196643, 62266334, 63373149, 62570953, 62624850, 62530384, 62753172, 62784306, 62880763, 62891528, 62713836, 55916126, 62259838, 62626786, 62451969, 62525835, 62853162, 22310619, 62543065, 62581410, 167949258, 62540834, 20837867, ...]",cell projection
GO:0009288,-0.641,-4.024,0.000e+00,0.000e+00,192,192,"[67153300, 64393106, 63578755, 52493103, 46374759, 63120114, 44786243, 63423978, 38387187, 43234476, 13586933, 63285308, 63543912, 37969809, 63436090, 48252443, 57751592, 62964712, 62062741, 62981262, 168017786, 62217709, 167570053, 166894504, 45075220, 51386722, 64167744, 51263942, 51837448, 64297496, 69175457, 65866265, 22143824, 62454720, 167732614, 167602424, 168130942, 167560438, 167355590, 167344702, 167246340, 166985137, 167799665, 67218229, 64780867, 63954167, 63277926, 62966887, 62682149, 62619244, 57528177, 49587153, 13589494, 43280069, 13571055, 167601916, 62560831, 62613597, 64917247, 166208736, 61924219, 62002315, 54589912, 62473024, 63356547, 62641817, 62699037, 62769590, 62263580, 63721498, 21809431, 62544664, 62303301, 3775835, 63636819, 55984017, 63842059, 63196643, 62266334, 63373149, 62570953, 62624850, 62530384, 62753172, 62784306, 62880763, 62891528, 62713836, 55916126, 62259838, 62626786, 62451969, 62525835, 62853162, 22310619, 62543065, 62581410, 167949258, 62540834, 20837867, ...]",bacterial-type flagellum
GO:0044422,-0.430,-2.854,0.000e+00,0.000e+00,497,371,"[63750366, 67091872, 165644696, 10689110, 58033748, 38785731, 2057470, 68051075, 209005, 65420876, 39551310, 66047049, 22073939, 69063475, 69008769, 40477281, 68014522, 69020279, 17052898, 4116244, 66872721, 65605812, 4343484, 64744508, 65043810, 4501661, 66591237, 64037691, 65380743, 68613446, 67882159, 23531183, 40393382, 55202336, 167251621, 166680682, 67297603, 8917842, 68607919, 28962319, 32811898, 18689322, 66350717, 166411480, 68877732, 56740485, 41367059, 38321268, 23586741, 42179199, 23713943, 30409136, 65324920, 29757812, 66665189, 166526328, 15445251, 57696478, 37966314, 65